# はじめに
本ノートブックはYomiToku Sagemaker Endpointの処理結果をもとに
- 様々な形式（CSV、Markdown、HTML、JSON）に変換する方法
- 変換したデータの解析結果を可視化する方法

についてのサンプルコードを提供します。

## 事前セットアップ
必要なライブラリのインストールを行います。

In [1]:
from yomitoku_client.parsers import parse_pydantic_model
from pprint import pprint
import pandas as pd
import json
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np


### サンプルデータの読み込み
`sample.json`はYomiToku SageMakerの出力結果をJSONファイルで保存したデータです。

In [9]:
with open("json/result_pdf.json", "r", encoding="utf-8") as f:
    sample_json =  json.load(f)

pprint(f"{sample_json}"[:500])

("{'result': [{'preprocess': {'angle': 0.0, 'angle_score': "
 "0.9999990463256836}, 'paragraphs': [{'box': [234, 74, 1424, 128], "
 "'contents': 'もっと知りたい!現在·未来のくらしと生活の情報誌', 'direction': 'horizontal', 'order': "
 "0, 'role': None, 'indent_level': None}, {'box': [248, 150, 1377, 452], "
 "'contents': '総務省', 'direction': 'horizontal', 'order': 1, 'role': "
 "'section_headings', 'indent_level': None}, {'box': [81, 486, 313, 521], "
 "'contents': '2024 年11月号', 'direction': 'horizontal', 'order': 2, 'role': "
 "None, 'indent_level")


### Parser とは？
- AWS SageMaker の Yomitoku サービスから返される生のJSON出力をPydantic モデルのラッパーオブジェクトに変換するためモデルです

主な機能：
1. 生のJSONデータを構造化されたPydanticオブジェクトに変換
2. データの検証とバリデーション
3. 各種フォーマットへの変換とエクスポートを実行でます。
4. データの可視化、読み取り結果の確認を容易にします。

In [10]:
# parse_dict: SageMaker Yomitokuの辞書形式の出力をPydanticモデルに変換
sample_data = parse_pydantic_model(sample_json)

## データエクスポート機能
### 様々な形式への変換とファイル保存

このセクションでは、OCR処理済みのデータを実用的な各種フォーマットに変換し、ファイルとして保存する方法を示します。

**対応フォーマット：**
- **CSV**: Excel等の表計算ソフトで開ける形式。データ分析に最適
- **Markdown**: GitHubやドキュメント作成に便利な軽量マークアップ形式
- **HTML**: ブラウザで直接表示可能。レポートやプレゼンテーションに活用
- **JSON**: プログラムで処理しやすい構造化データ。API連携やデータベース保存に最適

**引数:**
- `output_path`: 出力ファイルのパスを指定します。パスにフォルダが含まれる場合は指定フォルダに出力します。
- `mode`: 出力モード(`combine` or `separate`)を指定します。default: `combine`
    - `combine`モードは複数ページの読み取り結果を単一ファイルにまとめて出力
    - `separate`モードは複数ページをそれぞれのページに分割して出力
- `page_index`: 出力対象のページを指定します。(e.g. [0,1,2])
- `ignore_line_break`: 段落ごとの読み取り結果の改行を無視して出力

### CSV 形式で保存

In [4]:
# 読み取り結果をcsv形式で保存
sample_data.to_csv(output_path="sample.csv")

# demoフォルダ内に読み取り結果をcsv形式で保存. page番号を表すsuffix付き
#sample_data.to_csv(output_path="demo/sample.csv")

# 読み取り結果をページごとにcsv形式で保存（mode="separate"）
#sample_data.to_csv(output_path="sample.csv", mode="separate")

# 指定のページの読み取り結果をcsv形式で保存
#sample_data.to_csv(output_path="sample.csv", mode="separate", page_index=[0,2])
#sample_data.to_csv(output_path="sample.csv", mode="separate", page_index=1)

# 改行を無視してcsv形式で保存
#sample_data.to_csv(output_path="sample.csv", ignore_line_break=True)

['"TELEWORK\nWORK"\r\n\r\n"RK\nWORK TELEWORK"\r\n\r\n"テレワークのさらなる普及・定着に向け「テレワーク月間」を実施します!\nTELEWORK"\r\n\r\n"テレワークのさらなる普及·定着に向け\n「テレワーク月間」\nを実施します!"\r\n\r\n"総務省は、内閣官房、内閣府、デジタル庁、\n厚生労働省、経済産業省、国土交通省、観光庁、\n環境省、日本テレワーク協会、日本テレワーク\n学会と連携して、11月をテレワーク月間とし、\nテレワークのさらなる普及·定着に向けた各種\nイベント等を集中的に開催します。テレワーク\n月間期間中は駅構内やイベント会場等にポス\nターを掲出するほか、ホームページ(※)にて、テ\nレワーク実施団体·実施者の登録受付や、テレ\nワーク活用に関するトピック·コンテンツの掲\n載、各種イベントに係る情報発信等を行います。"\r\n\r\n令和6年度テレワーク月間ポスター\r\n\r\n文字サイズの食具\r\n\r\nテレワーク導入お役立ち情報\r\n\r\n※テレワーク月間ホームページ\r\n\r\nhttps://teleworkgekkan.go.jp/\r\n\r\n"働きたい、働き続けたいに、テレワーク\nテレワーク継続意向は80%超"\r\n\r\n2024\r\n\r\nデジタルテ\r\n\r\n"テレワーク月間\n関連イベント\n参加者募集中!"\r\n\r\n"テレワーク月間WEBサイトでは、\nテレワークの先行事例、テレワークに関する\nイベント-お役立ち情報を発信しています。"\r\n\r\n...\r\n\r\nテレワーク月時\r\n\r\n"今年度は育児·介護休業法の改正法が成立し、\n来年度以降は3歳未満の子を育てる社員がテレ\nワークを選択できるように措置を講ずることが、\n事業主の努力義務になります。"\r\n\r\n"総務省では、地方や中小企業等を含め、テレ\nワークの活用により、多様な働き手が活躍の機\n会を持てるよう、テレビ番組とのタイアップを\n通じて、テレワークの必要性を改めて実感でき\nるような情報発信を行います。また、全国の総\n合通信局等では、それぞれの地域における取組\nやテレワーク活用を進める先進企業を紹介する\n催しを実施しま

### Markdown形式で保存

In [5]:
# 読み取り結果をmd形式で保存
sample_data.to_markdown(output_path="sample.md")

# demoフォルダ内に読み取り結果をmd形式で保存. page番号を表すsuffix付き
#sample_data.to_markdown(output_path="demo/sample.md")

# 読み取り結果をページごとにmd形式で保存（mode="separate"）
#sample_data.to_markdown(output_path="sample.md", mode="separate")

# 指定のページの読み取り結果をmd形式で保存
#sample_data.to_markdown(output_path="sample.md", mode="separate", page_index=[0,2])
#sample_data.to_markdown(output_path="sample.md", mode="separate", page_index=1)

# 改行を無視してmd形式で保存
#sample_data.to_markdown(output_path="sample.md", ignore_line_break=True)

['もっと知りたい\\!現在·未来のくらしと生活の情報誌\n\n# 総務省\n\n2024 年11月号\n\nMinistry of<br>Internal Affairs and<br>Communications\n\nVol.287\n',
 'TELEWORK<br>WORK\n\nRK<br>WORK TELEWORK\n\nテレワークのさらなる普及・定着に向け「テレワーク月間」を実施します\\!<br>TELEWORK\n\n# テレワークのさらなる普及·定着に向け<br>「テレワーク月間」<br>を実施します\\!\n\n総務省は、内閣官房、内閣府、デジタル庁、<br>厚生労働省、経済産業省、国土交通省、観光庁、<br>環境省、日本テレワーク協会、日本テレワーク<br>学会と連携して、11月をテレワーク月間とし、<br>テレワークのさらなる普及·定着に向けた各種<br>イベント等を集中的に開催します。テレワーク<br>月間期間中は駅構内やイベント会場等にポス<br>ターを掲出するほか、ホームページ\\(※\\)にて、テ<br>レワーク実施団体·実施者の登録受付や、テレ<br>ワーク活用に関するトピック·コンテンツの掲<br>載、各種イベントに係る情報発信等を行います。\n\n令和6年度テレワーク月間ポスター\n\n文字サイズの食具\n\nテレワーク導入お役立ち情報\n\n※テレワーク月間ホームページ\n\nhttps://teleworkgekkan.go.jp/\n\n働きたい、働き続けたいに、テレワーク<br>テレワーク継続意向は80%超\n\n2024\n\nデジタルテ\n\nテレワーク月間<br>関連イベント<br>参加者募集中\\!\n\nテレワーク月間WEBサイトでは、<br>テレワークの先行事例、テレワークに関する<br>イベント\\-お役立ち情報を発信しています。\n\n...\n\nテレワーク月時\n\n今年度は育児·介護休業法の改正法が成立し、<br>来年度以降は3歳未満の子を育てる社員がテレ<br>ワークを選択できるように措置を講ずることが、<br>事業主の努力義務になります。\n\n総務省では、地方や中小企業等を含め、テレ<br>ワークの活用により、多様な働き手が活躍の機<br>会を持てるよう、テレビ番組とのタイアップを<br>

### HTML形式で保存

In [6]:
# 読み取り結果をhtml形式で保存
sample_data.to_html(output_path="sample.html")

# demoフォルダ内に読み取り結果をhtml形式で保存. page番号を表すsuffix付き
#sample_data.to_html(output_path="demo/sample.html")

# 読み取り結果をページごとにhtml形式で保存（mode="separate"）
#sample_data.to_html(output_path="sample.html", mode="separate")

# 指定のページの読み取り結果をhtml形式で保存
#sample_data.to_html(output_path="sample.html", mode="separate", page_index=[0,2])
#sample_data.to_html(output_path="sample.html", mode="separate", page_index=1)

# 改行を無視してhtml形式で保存
#sample_data.to_html(output_path="sample.html", ignore_line_break=True)

['<div><p>もっと知りたい!現在·未来のくらしと生活の情報誌</p>\n<h1>総務省</h1>\n<p>2024 年11月号</p>\n<p>Ministry of<br/>Internal Affairs and<br/>Communications</p>\n<p>Vol.287</p>\n</div>\n',
 '<div><p>TELEWORK<br/>WORK</p>\n<p>RK<br/>WORK TELEWORK</p>\n<p>テレワークのさらなる普及・定着に向け「テレワーク月間」を実施します!<br/>TELEWORK</p>\n<h1>テレワークのさらなる普及·定着に向け<br/>「テレワーク月間」<br/>を実施します!</h1>\n<p>総務省は、内閣官房、内閣府、デジタル庁、<br/>厚生労働省、経済産業省、国土交通省、観光庁、<br/>環境省、日本テレワーク協会、日本テレワーク<br/>学会と連携して、11月をテレワーク月間とし、<br/>テレワークのさらなる普及·定着に向けた各種<br/>イベント等を集中的に開催します。テレワーク<br/>月間期間中は駅構内やイベント会場等にポス<br/>ターを掲出するほか、ホームページ(※)にて、テ<br/>レワーク実施団体·実施者の登録受付や、テレ<br/>ワーク活用に関するトピック·コンテンツの掲<br/>載、各種イベントに係る情報発信等を行います。</p>\n<p>令和6年度テレワーク月間ポスター</p>\n<p>文字サイズの食具</p>\n<p>テレワーク導入お役立ち情報</p>\n<p>※テレワーク月間ホームページ</p>\n<p>https://teleworkgekkan.go.jp/</p>\n<p>働きたい、働き続けたいに、テレワーク<br/>テレワーク継続意向は80%超</p>\n<p>2024</p>\n<p>デジタルテ</p>\n<p>テレワーク月間<br/>関連イベント<br/>参加者募集中!</p>\n<p>テレワーク月間WEBサイトでは、<br/>テレワークの先行事例、テレワークに関する<br/>イベント-お役立ち情報を発信しています。</p>\n<p>...</p>\n<p>テレワーク月時</p>\n<p>今年度は育児·介護休業法の改正法が成立し、<br/>来

### JSON形式で保存

In [7]:
# 読み取り結果をjson形式で保存
sample_data.to_json(output_path="sample.json")

# demoフォルダ内に読み取り結果をjson形式で保存. page番号を表すsuffix付き
#sample_data.to_json(output_path="demo/sample.json")

# 読み取り結果をページごとにjson形式で保存（mode="separate"）
#sample_data.to_json(output_path="sample.json", mode="separate")

# 指定のページの読み取り結果をjson形式で保存
#sample_data.to_json(output_path="sample.json", mode="separate", page_index=[0,2])
#sample_data.to_json(output_path="sample.json", mode="separate", page_index=1)

# 改行を無視してjson形式で保存
#sample_data.to_json(output_path="sample.json", ignore_line_break=True)

[{'figures': [{'box': [1475, 178, 1581, 348],
    'caption': None,
    'decode': None,
    'direction': 'horizontal',
    'order': 3,
    'paragraphs': [{'box': [1471, 297, 1583, 345],
      'contents': 'MIC',
      'direction': 'horizontal',
      'indent_level': None,
      'order': 0,
      'role': None}],
    'role': 'logo'},
   {'box': [32, 605, 1624, 2281],
    'caption': None,
    'decode': None,
    'direction': 'horizontal',
    'order': 6,
    'paragraphs': [{'box': [72, 640, 480, 681],
      'contents': 'CONTENTS',
      'direction': 'horizontal',
      'indent_level': None,
      'order': 0,
      'role': None},
     {'box': [86, 936, 136, 971],
      'contents': 'P.6',
      'direction': 'horizontal',
      'indent_level': None,
      'order': 1,
      'role': None},
     {'box': [156, 863, 377, 1043],
      'contents': '地方のかがやき\n宮崎県\n都農町',
      'direction': 'horizontal',
      'indent_level': None,
      'order': 2,
      'role': None},
     {'box': [156, 1076, 892, 1120

## 可視化
このセクションでは、OCRで検出されたテキスト、図表、テーブルなどの要素を画像上に可視化する機能を紹介します。
これらの可視化機能により、OCR結果の品質確認や文書構造の理解が容易になります。

- **レイアウト構造のレンダリング**: 文書のレイアウト情報やレイアウトカテゴリを視覚的に確認
- **OCRの読み取り結果のレンダリング**：文書の読み取り結果の文字列情報や読み取り位置を確認

** 引数 **
- `image_path` : 読み取り結果の元画像のパスを指定(.png, .jpg, .bmp, .tif, .pdf)
- `mode` : 可視化対象の情報("layout" or "ocr") 
- `output_directory`: 可視化画像の主力先フォルダのパスを指定
- `page_index`; 可視化対象のページ番号
- `dpi`: 対象ファイルがPDFの場合は、画像に変換したときのDPIを指定する

### OCRの読み取り結果の可視化
OCRとは
- 文書内に存在する文字の座標情報の取得
- 文書化に存在する文字を認識し、文字情報を取得

In [6]:
# OCRの読み結果を画像上に可視化して保存
sample_data.visualize(
    image_path="sample/image.pdf",
    mode='ocr',
    page_index=None,
    output_directory="demo",
    dpi=200
)

# 先頭ページ(0)のみOCRの読み結果を画像上に可視化して保存
#sample_data.visualize(
#    image_path="sample/image.pdf",
#    mode='ocr',
#    page_index=0,
#    output_directory="demo",
#)

### レイアウト解析の結果の可視化
レイアウト解析とは
- 文書のレイアウト構造などの領域とカテゴリ(見出し、段落、図、表)の推定
- 表のセルの区切り位置やセルの座標、セル内部の文字情報の取得
- 文書レイアウトの読み取り順序を推定

In [9]:
# レイアウト解析の結果を画像上に可視化して保存
sample_data.visualize(
    image_path="sample/image.pdf",
    mode='layout',
    page_index=None,
    output_directory="demo",
    dpi=200
)

###  手書き文字画像の読み取り結果の可視化とマークダウン変換のサンプル

In [8]:
with open("json/tegaki.json", "r", encoding="utf-8") as f:
    tegaki_json =  json.load(f)

tegaki_data = parse_pydantic_model(tegaki_json)

tegaki_data.visualize(
    image_path="sample/tegaki.jpg",
    mode='ocr',
    page_index=None,
    output_directory="demo",
)

tegaki_data.to_markdown(output_path="tegaki.md")


['# 日本人移民\n\nNo\n\nDate\n\n「元年者」<br>ハワイ<br>「若松コロニー」カリフォルニア\n\n⇒ 劣悪な労働環境<br>黒人奴隷と同等の扱い\n\n明治政府 国辱的な外交問題→日本人労働者の海外渡航禁止\n\n松方デフレや地租改正による農村の窮乏 「渡米案内」\n\n→「官約移民」 ハワイ 砂糖耕地\n\nアメリカ本土にも日本人労働者\n\n排華移民法により中国人労働者の流入が停止\n\n→その代わりの労働力としての日本人移民\n\n第4次世界大戦 アメリカ本土で日系人排斥運動\n\n「日本人学童隔離事件」\n\n↓\n\n日米紳士協定」新規の日本人労働者の流入停止\n\n出稼ぎ志向→定住志向.\n\n「写真花嫁」<br>他の\n\n- 日本は帰化不能外国人」としてアジア系移民の子とひとくくりにされるのを嫌った\n\n    - 第1次世界大戦 \\(<br>9移民のアメリカ化、国民化が進展<br>軍隊 堆場としての役割<br>自発性\n\n戦うことが市民権取得の近道\n\n☆1924年 移民法<br>「排日移民法」\n\n→日本での言い方\n\nこつ実際は日本人だけをターゲットにしているわけたな\n\n移民の量的、質的制限の結実\n\n国民の選別=「選び捨て」政策の最先端をいっていたアメリカ<br>優秀な人のみ入国可\n\n# 第2次世界大戦\n\nマイノリティーの国民化\n\n日系アメリカ人の強制収容所、連行<br>446492部隊 ジレンマ\n']